In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
apiwrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=apiwrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model='llama2'))
retriver = vectordb.as_retriever()
retriver

/var/folders/1y/10sklkfx0tl41g0n9mf_cr5c0000gn/T/ipykernel_38037/1902097864.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model='llama2'))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10e429c40>, search_kwargs={})

In [32]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(
        retriever=retriver,
        name="Langsmith_Documentation",

        description='Search for information about langsmith. For any question about Langsmith, You must use this tool.'
)

In [30]:
print(retriver_tool.args)

{'query': {'description': 'query to look up in retriever', 'title': 'Query', 'type': 'string'}}


In [8]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arixiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arixiv = ArxivQueryRun(api_wrapper=arixiv_wrapper)
arixiv.name

'arxiv'

In [33]:
tools = [wiki,arixiv]

In [34]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama2")  # or "mistral", "gemma", etc.


In [35]:
from langchain import hub
prompt = hub.pull("hwchase17/react")  # ReAct format, compatible with all models
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

Agents
- The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [16]:
from langchain.agents import AgentType, initialize_agent

In [36]:
agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
                         ,agent_kwargs={"prompt": prompt},
)
agent.name

In [38]:
agent.run("What is compound interest?")

'Agent stopped due to iteration limit or time limit.'